# 데이터 읽어오기

In [5]:
import os, re 
import numpy as np
import tensorflow as tf
import glob

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [6]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


# 데이터 정제

## 정규표현식을 이용한 필터링

In [7]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


## 정제데이터 구축

In [8]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence) > 50 : continue
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> you saw her bathing on the roof <end>']

## 텐서로 변환

In [9]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   52    4 ...    0    0    0]
 [   2   17 3252 ...    0    0    0]
 [   2   38    7 ...    0    0    0]
 ...
 [   2    4  121 ...    0    0    0]
 [   2  259  194 ...    0    0    0]
 [   2    7   35 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fabf4d5f0d0>


## Padding

In [10]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])



[   2   52    4   94  339   65   56    9  953 6111    3    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0]
[  52    4   94  339   65   56    9  953 6111    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0]


## 데이터를 훈련데이터와 테스트데이터로 분리

In [11]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input, 
                                                          test_size=0.2,
                                                          random_state=42,) 

## 데이터셋  객체 만들기

In [12]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 27), (256, 27)), types: (tf.int32, tf.int32)>

## 전처리 요약
##### 정규표현식을 이용한 corpus 생성
##### tf.keras.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환
##### tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환

# 모델 생성

In [13]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [14]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 27, 7001), dtype=float32, numpy=
array([[[ 5.41681598e-04, -3.07244947e-04,  8.04283845e-05, ...,
         -2.36691543e-04,  2.04978045e-04, -6.57735218e-05],
        [ 8.25606810e-04, -5.57534804e-04, -2.18432484e-04, ...,
         -5.49558084e-04,  3.67457396e-04, -2.23899187e-04],
        [ 8.22804694e-04, -1.08765461e-03, -3.54898599e-04, ...,
         -9.36726399e-04,  2.66061921e-04, -3.54314870e-05],
        ...,
        [ 1.82848214e-03,  5.45008946e-03, -1.75667962e-03, ...,
         -2.78744963e-03,  2.03426680e-04,  3.86123988e-03],
        [ 1.85241061e-03,  5.48129948e-03, -1.73596677e-03, ...,
         -2.78727408e-03,  2.11591585e-04,  3.85455368e-03],
        [ 1.86552817e-03,  5.49529353e-03, -1.71105715e-03, ...,
         -2.78496882e-03,  2.18973830e-04,  3.84560856e-03]],

       [[ 5.41681598e-04, -3.07244947e-04,  8.04283845e-05, ...,
         -2.36691543e-04,  2.04978045e-04, -6.57735218e-05],
        [ 7.68541882e-04, -7.98606023e-04, -4.

In [15]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


# 학습

In [16]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
556/556 [==============================] - 144s 250ms/step - loss: 1.7042
Epoch 2/10
556/556 [==============================] - 148s 266ms/step - loss: 1.4320
Epoch 3/10
556/556 [==============================] - 149s 268ms/step - loss: 1.3480
Epoch 4/10
556/556 [==============================] - 149s 267ms/step - loss: 1.2788
Epoch 5/10
556/556 [==============================] - 149s 268ms/step - loss: 1.2207
Epoch 6/10
556/556 [==============================] - 149s 268ms/step - loss: 1.1698
Epoch 7/10
556/556 [==============================] - 149s 268ms/step - loss: 1.1225
Epoch 8/10
556/556 [==============================] - 149s 268ms/step - loss: 1.0783
Epoch 9/10
556/556 [==============================] - 149s 268ms/step - loss: 1.0361
Epoch 10/10
556/556 [==============================] - 149s 268ms/step - loss: 0.9957


# 모델 평가하기

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [18]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you , i love you <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> are you")

'<start> are you ready for the one ? <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> i am")

'<start> i am the walrus <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> who")

'<start> who s that chick ? who s that chick ? <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> I would")

'<start> i would not be flawless , but you know <end> '

In [23]:
generate_text(model, tokenizer, init_sentence="<start> yes")

'<start> yes , i m a voodoo child <end> '

In [24]:
generate_text(model, tokenizer, init_sentence="<start> by the way")

'<start> by the way you were <end> '

In [25]:
generate_text(model, tokenizer, init_sentence="<start> happy")

'<start> happy , free , free <end> '

In [26]:
generate_text(model, tokenizer, init_sentence="<start> good job")

'<start> good job , i m a <unk> <end> '